In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1662547440890_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql.functions import col, year, lit, udf, sha2, concat, explode, min, max
from pyspark.sql.types import StringType, ArrayType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

aws-cfn-bootstrap (2.0)
beautifulsoup4 (4.9.3)
boto (2.49.0)
click (8.0.3)
docutils (0.14)
jmespath (0.10.0)
joblib (1.1.0)
lockfile (0.11.0)
lxml (4.6.3)
mysqlclient (1.4.2)
nltk (3.6.5)
nose (1.3.4)
numpy (1.16.5)
pip (9.0.1)
py-dateutil (2.2)
pystache (0.5.4)
python-daemon (2.2.3)
python37-sagemaker-pyspark (1.4.1)
pytz (2021.3)
PyYAML (5.4.1)
regex (2021.10.21)
setuptools (28.8.0)
simplejson (3.2.0)
six (1.13.0)
tqdm (4.62.3)
wheel (0.29.0)
windmill (1.6)

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
You are using pip version 9.0.1, however version 22.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [5]:
sc.install_pypi_package("pygeodesy")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


You are using pip version 9.0.1, however version 22.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [6]:
output_path ="s3://patents-analytics/cleaned"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Cleaning Patent table

In [52]:
patent_path = "s3://patents-analytics/raw/patent.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
patent = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(patent_path).repartition(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
patent.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7991058

In [55]:
patent.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- number: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- abstract: string (nullable = true)
 |-- title: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- num_claims: integer (nullable = true)
 |-- filename: string (nullable = true)
 |-- withdrawn: integer (nullable = true)

In [56]:
patent_cleaned = patent.filter(col('withdrawn') == 0).filter(year(col('date'))>1999).select("id", "type", "date", "abstract", "title", "kind", "num_claims")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
patent_cleaned = patent_cleaned.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
patent_cleaned.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- abstract: string (nullable = true)
 |-- title: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- num_claims: integer (nullable = true)

### Test the data cleaning

In [60]:
assert patent_cleaned.count() > 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
assert patent_cleaned.agg({"date":"min"}).collect()[0]["min(date)"].year == 2000

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
assert patent_cleaned.filter(col("withdrawn")==1).count() == 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Write the cleaned data to S3 partitioned by year

In [71]:
patent_cleaned = patent_cleaned.withColumn('year', year(col('date')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
patent_cleaned.write.partitionBy("year").mode("overwrite").parquet(f"{output_path}/patent.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Cleaning the Assignee Table

In [3]:
assignee_path = "s3://patents-analytics/raw/assignee.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
assignee = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(assignee_path).repartition(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
assignee.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

540183

In [6]:
assignee.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- type: double (nullable = true)
 |-- name_first: string (nullable = true)
 |-- name_last: string (nullable = true)
 |-- organization: string (nullable = true)

### Replace the type with string

In [7]:
def convert_type_to_string(type):
    if type == 1:
        return 'unassigned'
    elif type == 12 or type == 2 or type == 13 or type == 3:
        return 'company'
    elif type == 4 or type == 5 or type == 14 or type == 15:
        return 'individual'
    elif type == 6 or type == 7 or type == 8 or type == 9 or type == 16 or type == 17 or type == 18 or type == 19:
        return 'government'
    else:
        'unknown'
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
convert_type_to_string_udf = udf(lambda type:convert_type_to_string(type), StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
assignee_cleaned = assignee.withColumn('type_string', convert_type_to_string_udf(col("type")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
assert assignee_cleaned.filter(col('type_string') == 'unknown').count() == 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
assignee_cleaned.filter(col('type_string').isNull()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+----------+---------+--------------------+-----------+
|                  id|type|name_first|name_last|        organization|type_string|
+--------------------+----+----------+---------+--------------------+-----------+
|f396335b-4315-4d2...|null|      null|     null|ROI Media LLC, a ...|       null|
|ea9e34d1-0610-417...|null|      null|     null|ZETA POTENTIAL IN...|       null|
|264b42fa-8451-4bc...|null|      null|     null|Dong Ahm Bio-Tech...|       null|
|6e620470-8766-421...|null|      null|     null|Shenzhen Joyhong ...|       null|
|0d083a92-6a48-4a5...|null|      null|     null| Lucemitek Co., Ltd.|       null|
|9037e1d8-c403-4f1...|null|      null|     null|Home Phone Tunes,...|       null|
|ec759e88-b2c5-4d1...|null|      null|     null|   Juicy Tours, Inc.|       null|
|ecd46ec5-fa4a-449...|null|      null|     null|Truplast Kunststo...|       null|
|bd4801df-0484-4ad...|null|      null|     null|         Nooreq Inc.|       null|
|1f30dece-956f-4

In [12]:
def infer_type(type_string, name_first, name_last, organization):
    if type_string:
        return type_string
    else:
        if organization:
            return 'company'
        elif name_first or name_last:
            return 'individual'
        else:
            return 'unknown'
        

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
infer_type_udf = udf(infer_type, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
assignee_cleaned = assignee_cleaned.withColumn('type_string_cleaned', infer_type_udf(col("type_string"), col("name_first"), col("name_last"), col("organization")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
assignee_cleaned.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+----------+---------+--------------------+-----------+-------------------+
|                  id|type|name_first|name_last|        organization|type_string|type_string_cleaned|
+--------------------+----+----------+---------+--------------------+-----------+-------------------+
|cf06cbad-b4f3-45f...| 2.0|      null|     null|Rainbow Display, ...|    company|            company|
|4f18552a-5db7-4b0...| 2.0|      null|     null|    LECO Corporation|    company|            company|
|24effb01-7bc0-43c...| 2.0|      null|     null| Sper Scientific Ltd|    company|            company|
|e47b6be9-98b3-46c...| 2.0|      null|     null|Amenex Associates...|    company|            company|
|91d19128-6b46-454...| 3.0|      null|     null|Living Technologi...|    company|            company|
+--------------------+----+----------+---------+--------------------+-----------+-------------------+
only showing top 5 rows

In [16]:
assignee_cleaned.filter(col("type_string").isNull()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+----------+---------+--------------------+-----------+-------------------+
|                  id|type|name_first|name_last|        organization|type_string|type_string_cleaned|
+--------------------+----+----------+---------+--------------------+-----------+-------------------+
|f396335b-4315-4d2...|null|      null|     null|ROI Media LLC, a ...|       null|            company|
|ea9e34d1-0610-417...|null|      null|     null|ZETA POTENTIAL IN...|       null|            company|
|264b42fa-8451-4bc...|null|      null|     null|Dong Ahm Bio-Tech...|       null|            company|
|6e620470-8766-421...|null|      null|     null|Shenzhen Joyhong ...|       null|            company|
|0d083a92-6a48-4a5...|null|      null|     null| Lucemitek Co., Ltd.|       null|            company|
|9037e1d8-c403-4f1...|null|      null|     null|Home Phone Tunes,...|       null|            company|
|ec759e88-b2c5-4d1...|null|      null|     null|   Juicy Tours, Inc.|       null| 

In [17]:
assignee_cleaned.filter(col("type_string_cleaned").isNull()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+----------+---------+------------+-----------+-------------------+
| id|type|name_first|name_last|organization|type_string|type_string_cleaned|
+---+----+----------+---------+------------+-----------+-------------------+
+---+----+----------+---------+------------+-----------+-------------------+

### Combine the column name_first, name_last, and organization

In [18]:
def combine_name(name_first, name_last, organization):
    if organization:
        return organization
    elif name_first and name_last:
        return f"{name_first} {name_last}"
    elif name_last:
        return name_last
    elif name_first:
        return name_first
    else:
        return null

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
combine_name_udf = udf(combine_name, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
assignee_cleaned = assignee_cleaned.withColumn('name', combine_name_udf(col("name_first"), col("name_last"), col("organization")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
assignee_cleaned.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+----------+---------+--------------------+-----------+-------------------+--------------------+
|                  id|type|name_first|name_last|        organization|type_string|type_string_cleaned|                name|
+--------------------+----+----------+---------+--------------------+-----------+-------------------+--------------------+
|cf06cbad-b4f3-45f...| 2.0|      null|     null|Rainbow Display, ...|    company|            company|Rainbow Display, ...|
|4f18552a-5db7-4b0...| 2.0|      null|     null|    LECO Corporation|    company|            company|    LECO Corporation|
|24effb01-7bc0-43c...| 2.0|      null|     null| Sper Scientific Ltd|    company|            company| Sper Scientific Ltd|
|e47b6be9-98b3-46c...| 2.0|      null|     null|Amenex Associates...|    company|            company|Amenex Associates...|
|91d19128-6b46-454...| 3.0|      null|     null|Living Technologi...|    company|            company|Living Technologi...|
|3390a121-64fd-4

In [22]:
assignee_cleaned.filter(col("name").isNull()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+----------+---------+------------+-----------+-------------------+----+
| id|type|name_first|name_last|organization|type_string|type_string_cleaned|name|
+---+----+----------+---------+------------+-----------+-------------------+----+
+---+----+----------+---------+------------+-----------+-------------------+----+

In [78]:
assignee_cleaned.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

540183

### Delete the unassigned 

In [23]:
assignee_cleaned = assignee_cleaned.filter((col('type_string_cleaned')!='unassigned'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
assignee_cleaned.filter((col('type_string_cleaned')=='unassigned') | (col('type_string_cleaned')=='unknown')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+----------+---------+------------+-----------+-------------------+----+
| id|type|name_first|name_last|organization|type_string|type_string_cleaned|name|
+---+----+----------+---------+------------+-----------+-------------------+----+
+---+----+----------+---------+------------+-----------+-------------------+----+

### Create unique ID based on the name and type_string_cleaned column

In [25]:
assignee_cleaned_with_ID = assignee_cleaned.groupBy("name", "type_string_cleaned").count().withColumn("new_id", sha2(concat(col("name"), col("type_string_cleaned")),256)).select(["name", "type_string_cleaned", "new_id"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
assignee_cleaned_with_ID.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+--------------------+
|                name|type_string_cleaned|              new_id|
+--------------------+-------------------+--------------------+
|   Medisol USA, Inc.|            company|6ebe7a88061c774f3...|
|         Expii, Inc.|            company|a8a7e11b2eb6da7cb...|
|     A.L. FILTER EAD|            company|e8dfa02ea03b1e34c...|
|Wound Engineering...|            company|a78e711e20df5c9c4...|
|M & B Plating Rac...|            company|0c06c9bfc48b0c904...|
|        Mikroelektro|            company|616256456a5df3735...|
|Sandblast Solutio...|            company|cbda75c71db098ddb...|
|             Isoform|            company|413c7c04309760552...|
|UP Management Gmb...|            company|801b25941581280e4...|
|Bayerische Motore...|            company|a0403269fbface752...|
|Rockwell Automoti...|            company|e896b40d6b47bb4e0...|
|Constad Transfer,...|            company|cb90ea15069bab4ef...|
|Yellowtuna Holdin...|            compan

In [27]:
assignee_cleaned_with_ID = assignee_cleaned.join(assignee_cleaned_with_ID, (assignee_cleaned.name == assignee_cleaned_with_ID.name) & (assignee_cleaned.type_string_cleaned == assignee_cleaned_with_ID.type_string_cleaned)).drop(assignee_cleaned_with_ID.name).drop(assignee_cleaned_with_ID.type_string_cleaned)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
assignee_cleaned_with_ID = assignee_cleaned_with_ID.select(["id", "name", col("type_string_cleaned").alias("type"), "new_id"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
assignee_cleaned_with_ID.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

540181

In [31]:
assignee_cleaned_with_ID.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- new_id: string (nullable = true)

### Write the cleaned assignee data to S3

In [32]:
assignee_cleaned_with_ID.write.mode("overwrite").parquet(f"{output_path}/assignee.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Cleaning the Location Table

In [7]:
location_path = "s3://patents-analytics/raw/location.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
location = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(location_path).repartition(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
location.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+-------+--------+---------+------+----------+-----------+
|                  id|                city|state|country|latitude|longitude|county|state_fips|county_fips|
+--------------------+--------------------+-----+-------+--------+---------+------+----------+-----------+
|e10f3d8a-cb8e-11e...|Saint Laurant du Var| null|     FR| 46.2621| -0.21006|  null|      null|       null|
|02b3429f-cb90-11e...|            Zornhcim| null|     DE|    null|     null|  null|      null|       null|
|bfe59c31-cb8e-11e...|       Doddanankundi| null|     IN|    null|     null|  null|      null|       null|
|8147f6e3-cb8e-11e...|        Stella-Plage| null|     FR| 50.4799|  1.57714|  null|      null|       null|
|fa670c89-cb8e-11e...|           Naremburn| null|     AU|-33.8171|  151.202|  null|      null|       null|
+--------------------+--------------------+-----+-------+--------+---------+------+----------+-----------+
only showing top 5 rows

In [10]:
location.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

242926

In [11]:
list_of_states_in_us = ["AL",  "AK",  "AS",  "AZ",  "AR",  "CA",  "CO",  "CT",  "DE",  "DC",  "FL",  "GA",  "GU",  "HI",  "ID",  "IL",  "IN",  "IA",  "KS",  "KY",  "LA",  "ME",  "MD",  "MA",  "MI",  "MN",  "MS",  "MO",  "MT",  "NE",  "NV",  "NH",  "NJ",  "NM",  "NY",  "NC",  "ND",  "MP",  "OH",  "OK",  "OR",  "PA",  "PR",  "RI",  "SC",  "SD",  "TN",  "TX",  "UT",  "VT",  "VA",  "VI",  "WA",  "WV",  "WI",  "WY"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
def map_us_state_to_country(state):
    list_of_states_in_us = ["AL",  "AK",  "AS",  "AZ",  "AR",  "CA",  "CO",  "CT",  "DE",  "DC",  "FL",  "GA",  "GU",  "HI",  "ID",  "IL",  "IN",  "IA",  "KS",  "KY",  "LA",  "ME",  "MD",  "MA",  "MI",  "MN",  "MS",  "MO",  "MT",  "NE",  "NV",  "NH",  "NJ",  "NM",  "NY",  "NC",  "ND",  "MP",  "OH",  "OK",  "OR",  "PA",  "PR",  "RI",  "SC",  "SD",  "TN",  "TX",  "UT",  "VT",  "VA",  "VI",  "WA",  "WV",  "WI",  "WY"]
    if state and state in list_of_states_in_us:
        return 'US'
    else:
        return None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
map_us_state_to_country_udf = udf(map_us_state_to_country, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
location = location.withColumn("derived_country_based_on_state", map_us_state_to_country_udf(col("state"))).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Load the country shapes data to transform latitude and longitude to country

In [18]:
country_shapes_path = "s3://patents-analytics/raw/shapes_simplified_low.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
country_shapes = spark.read.options(multiline="True").json(country_shapes_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
country_shapes.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- features: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- geometry: struct (nullable = true)
 |    |    |    |-- coordinates: array (nullable = true)
 |    |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |-- properties: struct (nullable = true)
 |    |    |    |-- geoNameId: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)

In [21]:
country_shapes.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [22]:
country_shapes2=country_shapes.select(explode("features"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
country_shapes2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- col: struct (nullable = true)
 |    |-- geometry: struct (nullable = true)
 |    |    |-- coordinates: array (nullable = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |-- type: string (nullable = true)
 |    |-- properties: struct (nullable = true)
 |    |    |-- geoNameId: string (nullable = true)
 |    |-- type: string (nullable = true)

In [24]:
country_shapes3 = country_shapes2.select("col.properties.geoNameId", "col.geometry.coordinates", "col.geometry.type")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
country_shapes4 = country_shapes3.select("geoNameId", "type", explode("coordinates").alias("polygon"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
country_shapes4 = country_shapes4.withColumn("polygon_id", sha2(col("polygon").cast("string"),256)).select("geoNameId", "type", "polygon", "polygon_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
country_shapes4.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geoNameId: string (nullable = true)
 |-- type: string (nullable = true)
 |-- polygon: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- polygon_id: string (nullable = true)

In [28]:
country_shapes5 = country_shapes4.select("geoNameId", "type", explode("polygon").alias("polygon"), "polygon_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
country_polygon = country_shapes5.filter(col("type")=='Polygon')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
country_polygon.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geoNameId: string (nullable = true)
 |-- type: string (nullable = true)
 |-- polygon: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- polygon_id: string (nullable = true)

In [31]:
country_multi_polygon= country_shapes5.filter(col("type")=='MultiPolygon')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
country_multi_polygon.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geoNameId: string (nullable = true)
 |-- type: string (nullable = true)
 |-- polygon: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- polygon_id: string (nullable = true)

In [33]:
country_multi_polygon2 = country_multi_polygon.select("geoNameId", "type", explode("polygon").alias("polygon"), "polygon_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
country_multi_polygon2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geoNameId: string (nullable = true)
 |-- type: string (nullable = true)
 |-- polygon: string (nullable = true)
 |-- polygon_id: string (nullable = true)

In [35]:
import json
def convert_string_to_array(str):
    result = json.loads(str)
    return result

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
convert_string_to_array_udf = udf(convert_string_to_array, ArrayType(StringType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
country_multi_polygon2= country_multi_polygon2.withColumn("new_polygon", convert_string_to_array_udf(col("polygon")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
country_multi_polygon2 = country_multi_polygon2.select("geoNameId", "type", col("new_polygon").alias("polygon"), "polygon_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
country_multi_polygon2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geoNameId: string (nullable = true)
 |-- type: string (nullable = true)
 |-- polygon: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- polygon_id: string (nullable = true)

In [40]:
country_shapes5 = country_polygon.union(country_multi_polygon2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
country_shapes5.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geoNameId: string (nullable = true)
 |-- type: string (nullable = true)
 |-- polygon: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- polygon_id: string (nullable = true)

In [42]:
country_shapes6 = country_shapes5.withColumn("latitude", col("polygon").getItem(1)).withColumn("longitude", col("polygon").getItem(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
country_shapes6.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------+----------------+--------------------+--------+---------+
|geoNameId|   type|         polygon|          polygon_id|latitude|longitude|
+---------+-------+----------------+--------------------+--------+---------+
|    49518|Polygon| [29.96, -2.327]|d7bdba6946ced25d0...|  -2.327|    29.96|
|    49518|Polygon|[29.919, -2.703]|d7bdba6946ced25d0...|  -2.703|   29.919|
|    49518|Polygon|[29.724, -2.819]|d7bdba6946ced25d0...|  -2.819|   29.724|
|    49518|Polygon|[29.438, -2.798]|d7bdba6946ced25d0...|  -2.798|   29.438|
|    49518|Polygon| [29.371, -2.84]|d7bdba6946ced25d0...|   -2.84|   29.371|
|    49518|Polygon|[29.326, -2.654]|d7bdba6946ced25d0...|  -2.654|   29.326|
|    49518|Polygon| [29.15, -2.592]|d7bdba6946ced25d0...|  -2.592|    29.15|
|    49518|Polygon|[29.062, -2.602]|d7bdba6946ced25d0...|  -2.602|   29.062|
|    49518|Polygon| [29.04, -2.745]|d7bdba6946ced25d0...|  -2.745|    29.04|
|    49518|Polygon| [28.897, -2.66]|d7bdba6946ced25d0...|   -2.66|   28.897|

In [44]:
country_shapes6 = country_shapes6.select("geoNameId", "type", "polygon_id", "latitude", "longitude")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
country_shapes6.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geoNameId: string (nullable = true)
 |-- type: string (nullable = true)
 |-- polygon_id: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)

### Load country info path which has the mapping between geoNameId and the country name

In [15]:
country_info_path = "s3://patents-analytics/raw/countryInfo.txt"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
country_info = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(country_info_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
country_info.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ISO: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- ISO-Numeric: integer (nullable = true)
 |-- fips: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Capital: string (nullable = true)
 |-- Area(in sq km): double (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Continent: string (nullable = true)
 |-- tld: string (nullable = true)
 |-- CurrencyCode: string (nullable = true)
 |-- CurrencyName: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Postal Code Format: string (nullable = true)
 |-- Postal Code Regex: string (nullable = true)
 |-- Languages: string (nullable = true)
 |-- geonameid: integer (nullable = true)
 |-- neighbours: string (nullable = true)
 |-- EquivalentFipsCode: string (nullable = true)

In [49]:
country_shapes7 = country_shapes6.join(country_info, (country_shapes6.geoNameId == country_info.geonameid)).drop(country_info.geonameid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
country_shapes7.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geoNameId: string (nullable = true)
 |-- type: string (nullable = true)
 |-- polygon_id: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- ISO: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- ISO-Numeric: integer (nullable = true)
 |-- fips: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Capital: string (nullable = true)
 |-- Area(in sq km): double (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Continent: string (nullable = true)
 |-- tld: string (nullable = true)
 |-- CurrencyCode: string (nullable = true)
 |-- CurrencyName: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Postal Code Format: string (nullable = true)
 |-- Postal Code Regex: string (nullable = true)
 |-- Languages: string (nullable = true)
 |-- neighbours: string (nullable = true)
 |-- EquivalentFipsCode: string (nullable = true)

In [51]:
country_shapes7=country_shapes7.select("geoNameId", "type", "polygon_id", col('latitude').cast('double').alias('latitude'), col('longitude').cast('double').alias('longitude'), "ISO","Country","Continent")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
country_shapes7.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------+--------------------+--------+---------+---+-------+---------+
|geoNameId|   type|          polygon_id|latitude|longitude|ISO|Country|Continent|
+---------+-------+--------------------+--------+---------+---+-------+---------+
|    49518|Polygon|d7bdba6946ced25d0...|  -2.327|    29.96| RW| Rwanda|       AF|
|    49518|Polygon|d7bdba6946ced25d0...|  -2.703|   29.919| RW| Rwanda|       AF|
|    49518|Polygon|d7bdba6946ced25d0...|  -2.819|   29.724| RW| Rwanda|       AF|
|    49518|Polygon|d7bdba6946ced25d0...|  -2.798|   29.438| RW| Rwanda|       AF|
|    49518|Polygon|d7bdba6946ced25d0...|   -2.84|   29.371| RW| Rwanda|       AF|
|    49518|Polygon|d7bdba6946ced25d0...|  -2.654|   29.326| RW| Rwanda|       AF|
|    49518|Polygon|d7bdba6946ced25d0...|  -2.592|    29.15| RW| Rwanda|       AF|
|    49518|Polygon|d7bdba6946ced25d0...|  -2.602|   29.062| RW| Rwanda|       AF|
|    49518|Polygon|d7bdba6946ced25d0...|  -2.745|    29.04| RW| Rwanda|       AF|
|    49518|Polyg

In [53]:
country_shapes7.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geoNameId: string (nullable = true)
 |-- type: string (nullable = true)
 |-- polygon_id: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- ISO: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Continent: string (nullable = true)

In [54]:
country_shapes7.write.option("header",True).mode("overwrite").csv(f"{output_path}/country_shapes.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
country_shapes7 = spark.read.options(header='True', delimiter=',', inferSchema='True', escape="\"", multiline='True').csv(f"{output_path}/country_shapes.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
country_shapes7.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geoNameId: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- polygon_id: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- ISO: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Continent: string (nullable = true)

In [20]:
country_shapes_min_max = country_shapes7.groupBy("polygon_id").agg(min("latitude").alias('latitude_min'), max("latitude").alias("latitude_max"), min("longitude").alias("longitude_min"), max("longitude").alias("longitude_max"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
country_shapes_min_max.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- polygon_id: string (nullable = true)
 |-- latitude_min: double (nullable = true)
 |-- latitude_max: double (nullable = true)
 |-- longitude_min: double (nullable = true)
 |-- longitude_max: double (nullable = true)

In [22]:
country_shapes_min_max.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+------------+-------------+-------------+
|          polygon_id|latitude_min|latitude_max|longitude_min|longitude_max|
+--------------------+------------+------------+-------------+-------------+
|e281d5c3cf33ce407...|      23.977|      23.979|       38.186|       38.194|
|c17e7dda9babc5c58...|      60.451|      60.476|       21.073|        21.12|
|b132c41804b5213b6...|      -0.058|      -0.056|      103.477|      103.479|
|770d0e02b67db2c20...|      10.981|      11.005|      125.618|      125.653|
|64cf965a101f70214...|      34.885|      34.889|      126.105|      126.109|
|3c235f5598efa6f69...|       34.74|      34.756|      126.049|      126.061|
|623bdae70cf2fdc15...|      34.446|      34.464|      126.958|      126.966|
|675a5b43907eb925a...|      38.207|      38.215|       127.04|      127.046|
|f42b7b939d8706f4d...|      36.609|      36.614|      125.541|      125.547|
|271b808f09a92fdd4...|     -20.381|     -20.337|      148.928|       148.94|

In [23]:
country_shapes_broadcast = spark.sparkContext.broadcast(country_shapes7.collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
country_shapes_min_max_broadcast = spark.sparkContext.broadcast(country_shapes_min_max.collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
from pygeodesy.sphericalNvector import LatLon
def get_country_based_on_latitude_longitude(latitude, longitude):
    country_shapes_values = country_shapes_broadcast.value
    country_shapes_min_max_values = country_shapes_min_max_broadcast.value
    
    if latitude and longitude:
        filtered_polygon = [
            polygon.polygon_id
            for polygon in country_shapes_min_max_values
            if (
                polygon.latitude_min <= latitude
                and polygon.latitude_max >= latitude
                and polygon.longitude_min <= longitude
                and polygon.longitude_max >= longitude
            )
        ]
        filtered_country_shapes_values = [
            p for p in country_shapes_values if p.polygon_id in filtered_polygon
        ]
        total_polygons = len(filtered_country_shapes_values)
    
        target = LatLon(latitude, longitude)
        polygon = []
        prev_polygon_id = None
        prev_country_id = None
        for idx, point in enumerate(filtered_country_shapes_values):
            if idx == total_polygons - 1:
                if len(polygon) > 0:
                    try:
                        if target.isenclosedBy(polygon):
                            return point.ISO
                    except:
                        print("The length of polygon is ", len(polygon))
                else:
                    return None
            elif point.polygon_id == prev_polygon_id:
                polygon.append(LatLon(point.latitude, point.longitude))
            else:
                if len(polygon) > 0:
                    try:
                        if target.isenclosedBy(polygon):
                            return prev_country_id
                    except:
                        print("The length of polygon is ", len(polygon))
                else:
                    polygon = []
                    polygon.append(LatLon(point.latitude, point.longitude))
            prev_polygon_id = point.polygon_id
            prev_country_id = point.ISO
    else:
        return None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
get_country_based_on_latitude_longitude_udf = udf(get_country_based_on_latitude_longitude, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
location.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- county: string (nullable = true)
 |-- state_fips: double (nullable = true)
 |-- county_fips: integer (nullable = true)
 |-- derived_country_based_on_state: string (nullable = true)

In [28]:
location_with_latlon = location.filter(col("country").isNull() & col("latitude").isNotNull())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
location_with_latlon.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

167

In [30]:
location_rest = location.filter(~(col("country").isNull() & col("latitude").isNotNull()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
location_rest = location_rest.withColumn("derived_country_based_on_latlon", lit(None))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
location_rest.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

242759

In [33]:
location_with_latlon = location_with_latlon.withColumn("derived_country_based_on_latlon", get_country_based_on_latitude_longitude_udf(col('latitude'), col('longitude'))).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
location_with_latlon.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+-----+-------+--------+---------+------+----------+-----------+------------------------------+-------------------------------+
|                  id|       city|state|country|latitude|longitude|county|state_fips|county_fips|derived_country_based_on_state|derived_country_based_on_latlon|
+--------------------+-----------+-----+-------+--------+---------+------+----------+-----------+------------------------------+-------------------------------+
|cb55c189-09be-11e...|     Tustin|   CA|   null| 33.7335| -117.808|  null|      null|       null|                            US|                             US|
|1f9d1117-cb8f-11e...|   Broadway|   VA|   null| 38.6076| -78.8031|  null|      null|       null|                            US|                             US|
|44f21ef1-cb8f-11e...|      Macao| null|   null| 39.6086| -7.97485|  null|      null|       null|                          null|                             PT|
|7c989497-cb8e-11e...| Alamogordo|

In [35]:
location_combined = location_rest.union(location_with_latlon)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
def consolidate_country(country, derived_country_based_on_latlon, derived_country_based_on_state):
    if country:
        return country
    elif derived_country_based_on_latlon:
        return derived_country_based_on_latlon
    elif derived_country_based_on_state:
        return derived_country_based_on_state
    else:
        return None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
consolidate_country_udf = udf(consolidate_country, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
location_combined = location_combined.withColumn("final_country", consolidate_country_udf(col("country"), col("derived_country_based_on_latlon"), col("derived_country_based_on_state")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
location_combined.filter(col('final_country').isNull()).show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+-------+--------+---------+------+----------+-----------+------------------------------+-------------------------------+-------------+
|                  id|                city|state|country|latitude|longitude|county|state_fips|county_fips|derived_country_based_on_state|derived_country_based_on_latlon|final_country|
+--------------------+--------------------+-----+-------+--------+---------+------+----------+-----------+------------------------------+-------------------------------+-------------+
|36b3ac7a-cb8f-11e...|       Mountain View|    C|   null|    null|     null|  null|      null|       null|                          null|                           null|         null|
|91f9637a-cb8e-11e...|            Roterdam| null|   null|    null|     null|  null|      null|       null|                          null|                           null|         null|
|014c81d4-cb8f-11e...|Terrassa (Barcelona)| null|   null|    null|     null|  nu

In [76]:
location_combined.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

242926

In [45]:
location_combined2 = location_combined.join(country_info, (location_combined.final_country == country_info.ISO),"left").drop(location.country)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
location_combined2.filter(col("Country").isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3387

In [47]:
location_combined2.filter(col("Country").isNull()).groupby("final_country").count().orderBy(col("count").desc()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----+
|final_country|count|
+-------------+-----+
|           SU| 1133|
|           DT|  700|
|           JA|  276|
|           EN|  199|
|           UK|  176|
|           YU|  172|
|           un|  137|
|           SW|  125|
|           DD|  123|
|         null|   53|
|           SF|   37|
|           OE|   35|
|           PO|   21|
|           WA|   19|
|           NY|   14|
|           EI|   11|
|           DL|   10|
|           FL|   10|
|           TX|    8|
|           CT|    8|
+-------------+-----+
only showing top 20 rows

In [48]:
data = [('SU', 'Soviet Union', 'EU'),('DT', 'Germany', 'EU'),('JA', 'Japan', 'AS'),('EN', 'United Kingdom', 'EU'),('UK', 'United Kingdom', 'EU'),('YU','Yugoslavia', 'EU'), ('SW','Sweden', 'EU'), ('DD','Germany', 'EU')]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
columns = ["code", "country_based_on_code", "continent_based_on_code"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
additional_code = spark.createDataFrame(data=data, schema=columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
additional_code.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- code: string (nullable = true)
 |-- country_based_on_code: string (nullable = true)
 |-- continent_based_on_code: string (nullable = true)

In [54]:
location_combined3 = location_combined2.join(additional_code, location_combined2.final_country == additional_code.code, "left").drop(additional_code.code)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
location_combined3.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

242926

In [56]:
def consolidate_place_col(place, place_based_on_code):
    if place:
        return place
    if place_based_on_code:
        return place_based_on_code
    return None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
consolidate_place_col_udf = udf(consolidate_place_col, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
location_combined4 = location_combined3.withColumn("new_country", consolidate_place_col_udf("Country","country_based_on_code"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
location_combined5 = location_combined4.withColumn("new_continent", consolidate_place_col_udf("Continent","continent_based_on_code"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
location_combined5.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- county: string (nullable = true)
 |-- state_fips: double (nullable = true)
 |-- county_fips: integer (nullable = true)
 |-- derived_country_based_on_state: string (nullable = true)
 |-- derived_country_based_on_latlon: string (nullable = true)
 |-- final_country: string (nullable = true)
 |-- ISO: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- ISO-Numeric: integer (nullable = true)
 |-- fips: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Capital: string (nullable = true)
 |-- Area(in sq km): double (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Continent: string (nullable = true)
 |-- tld: string (nullable = true)
 |-- CurrencyCode: string (nullable = true)
 |-- CurrencyName: string (nullable = true)
 |-- Phone: string (nullable = 

In [62]:
location_combined5.filter(col('new_country').isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

483

In [63]:
location_combined6 = location_combined5.select("id", "new_country", "new_continent")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create unique ID for country

In [64]:
location_with_country_id = location_combined6.groupBy("new_country").count().withColumn("new_id", sha2(col("new_country"),256)).select(["new_country","new_id"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
location_with_country_id.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- new_country: string (nullable = true)
 |-- new_id: string (nullable = true)

In [66]:
location_with_country_id.filter(col("new_country").isNull()).show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+
|new_country|new_id|
+-----------+------+
|       null|  null|
+-----------+------+

In [67]:
location_combined7 = location_combined6.join(location_with_country_id, (location_with_country_id.new_country == location_combined6.new_country),"left").drop(location_with_country_id.new_country)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
location_combined7.filter(col("new_id").isNotNull()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+-----------+--------------------+
|                  id|new_continent|new_country|              new_id|
+--------------------+-------------+-----------+--------------------+
|7674c183-cb8f-11e...|           EU|    Albania|1af00b874ff318302...|
|9fceef4b-cbae-11e...|           EU|    Albania|1af00b874ff318302...|
|9940293b-cb90-11e...|           EU|    Albania|1af00b874ff318302...|
|cf759404-cb90-11e...|           EU|    Albania|1af00b874ff318302...|
|9970d12b-cbae-11e...|           EU|    Albania|1af00b874ff318302...|
|cb4161f3-09be-11e...|           EU|    Albania|1af00b874ff318302...|
|1edbbf10-cb91-11e...|           EU|    Albania|1af00b874ff318302...|
|8b0b08a1-cbae-11e...|           EU|    Albania|1af00b874ff318302...|
|a50da84f-cb8f-11e...|           EU|    Albania|1af00b874ff318302...|
|77b836a0-cbae-11e...|           EU|    Albania|1af00b874ff318302...|
|af7951c2-cb8f-11e...|           EU|    Albania|1af00b874ff318302...|
|8ddf72aa-cb8f-11e..

In [74]:
location_combined8 = location_combined7.select("id", "new_id", col("new_country").alias("Country"), col("new_continent").alias("Continent"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
location_combined8.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

242926

In [76]:
location_combined9 = location_combined8.filter(col("Country").isNotNull())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
location_combined9.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

242443

In [80]:
location_combined9.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- new_id: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Continent: string (nullable = true)

In [81]:
location_combined9.dropDuplicates(["new_id", "Country", "Continent"]) \
    .groupby(['Country']) \
    .count() \
    .where('count > 1') \
    .sort('count', ascending=False) \
    .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+
|Country|count|
+-------+-----+
+-------+-----+

### Write the cleaned Location to S3

In [82]:
location_combined9.write.mode("overwrite").parquet(f"{output_path}/location.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Cleaning the WIPO Table

In [2]:
wipo_path = "s3://patents-analytics/raw/wipo.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
wipo = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(wipo_path).repartition(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
wipo.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- patent_id: integer (nullable = true)
 |-- field_id: double (nullable = true)
 |-- sequence: integer (nullable = true)

In [5]:
wipo.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10658264

In [6]:
wipo.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(patent_id=5003481, field_id=32.0, sequence=0), Row(patent_id=9219777, field_id=6.0, sequence=1), Row(patent_id=9294431, field_id=6.0, sequence=1), Row(patent_id=6262715, field_id=6.0, sequence=0), Row(patent_id=7838201, field_id=17.0, sequence=1)]

### Filter sequence = 0

In [15]:
wipo = wipo.filter(col("sequence")==0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
assert wipo.count()>0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
wipo.filter(col("sequence")>0).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+
|patent_id|field_id|
+---------+--------+
+---------+--------+

In [18]:
wipo = wipo.select("patent_id", "field_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Write cleaned WIPO table to S3

In [21]:
wipo.write.mode("overwrite").parquet(f"{output_path}/wipo.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Clean the WIPO Field

In [146]:
wipo_field_path = "s3://patents-analytics/raw/wipo_field.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
wipo_field = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(wipo_field_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
wipo_field.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

70

In [156]:
wipo_field.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- sector_title: string (nullable = true)
 |-- field_title: string (nullable = true)

In [157]:
wipo_field.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+--------------------+
| id|        sector_title|         field_title|
+---+--------------------+--------------------+
|  1|Electrical engine...|Electrical machin...|
| 10|         Instruments|         Measurement|
| 11|         Instruments|Analysis of biolo...|
| 12|         Instruments|             Control|
| 13|         Instruments|  Medical technology|
+---+--------------------+--------------------+
only showing top 5 rows

In [158]:
wipo_field = wipo_field.filter(~col("id").startswith("D"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [159]:
wipo_field.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

36

In [161]:
assert wipo_field.filter(col("id").startswith("D")).count() ==0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Write the cleaned WIPO_field table to S3

In [162]:
wipo_field.write.mode("overwrite").parquet(f"{output_path}/wipo_field.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Cleaning the Inventor Table

In [83]:
inventor_path = "s3://patents-analytics/raw/inventor.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
inventor = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(inventor_path).repartition(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
inventor.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- name_first: string (nullable = true)
 |-- name_last: string (nullable = true)
 |-- male_flag: double (nullable = true)
 |-- attribution_status: integer (nullable = true)

In [86]:
inventor.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5363671

### Combine the first name and last name to name

In [87]:
def combine_name(name_first, name_last):
    if name_first and name_last:
        return f"{name_first} {name_last}"
    elif name_last:
        return name_last
    elif name_first:
        return name_first
    else:
        return None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
combine_name_udf = udf(combine_name, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
inventor = inventor.withColumn("name", combine_name_udf(col("name_first"),col("name_last")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
inventor.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+-----------+---------+------------------+--------------------+
|                  id|      name_first|  name_last|male_flag|attribution_status|                name|
+--------------------+----------------+-----------+---------+------------------+--------------------+
|fl:ka_ln:ruedinger-1|  Katrina Lauren|  Ruedinger|      0.0|                 1|Katrina Lauren Ru...|
|78c6413a-3b0f-11e...|Sascha Christoph|   Hrastnik|      1.0|                 1|Sascha Christoph ...|
|   fl:ru_ln:scholz-1|          Rudolf|     Scholz|      1.0|                 1|       Rudolf Scholz|
|fl:pa_ln:kirkpatr...|         Paul A.|Kirkpatrick|      1.0|                 1| Paul A. Kirkpatrick|
|     fl:ak_ln:baba-3|           Akira|       Baba|      1.0|                 1|          Akira Baba|
+--------------------+----------------+-----------+---------+------------------+--------------------+
only showing top 5 rows

In [91]:
inventor = inventor.withColumn("type", lit("individual"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
inventor = inventor.select("id","name", "type")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
inventor.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----------+
|                  id|                name|      type|
+--------------------+--------------------+----------+
|   fl:st_ln:benoit-4|        Steve Benoit|individual|
|fl:ka_ln:shirahas...|    Kazuo Shirahashi|individual|
| fl:da_ln:roberts-19|Daniel Neville Ro...|individual|
|fl:al_ln:shepherd-12|Allen S. Shepherd...|individual|
| fl:ro_ln:herrmann-8|  Robert P. Herrmann|individual|
+--------------------+--------------------+----------+
only showing top 5 rows

In [94]:
assert inventor.filter(col("type") != "individual").count() == 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
inventor_cleaned_with_ID = inventor.groupBy(["name","type"]).count().withColumn("new_id", sha2(concat(col("name"), col("type")),256)).select(["name", "new_id"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
inventor_cleaned_with_ID = inventor.join(inventor_cleaned_with_ID, inventor.name == inventor_cleaned_with_ID.name).drop(inventor_cleaned_with_ID.name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
inventor_cleaned_with_ID.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+--------------------+
|                  id|      type|                name|              new_id|
+--------------------+----------+--------------------+--------------------+
|   fl:a._ln:george-1|individual|    A. Chacko George|73b561e3245e37e27...|
|    fl:a._ln:craig-1|individual|       A. Grey Craig|8b1472e8c936a83ae...|
|  fl:al_ln:azarbar-2|individual|   ALI KIYAN AZARBAR|fda85794f8228a542...|
|fl:al_ln:gendrinb...|individual|ALINE GENDRIN BRO...|22c7a91733aeaeae3...|
|  fl:an_ln:speidel-6|individual|       ANDRE SPEIDEL|515a4ac3c6c1001d1...|
|1003217f-9466-11e...|individual|   ASHLEY L. JANSZEN|96fe198ef25033b8c...|
|    fl:aa_ln:egier-1|individual| Aaron Charles Egier|782bee9008eaaabf0...|
|  fl:aa_ln:schilpp-1|individual|   Aaron Dee Schilpp|c2843d342275553e2...|
| fl:aa_ln:anderson-1|individual| Aaron Ross Anderson|9f7817b8207be49e8...|
|   fl:aa_ln:patel-18|individual|        Aayush Patel|33eff47bca64985bb...|
|    fl:ab_l

### Write cleaned inventor table to S3

In [101]:
inventor_cleaned_with_ID.write.mode("overwrite").parquet(f"{output_path}/inventor.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Cleaning the Patent Assignee Table

In [47]:
patent_assignee_path = "s3://patents-analytics/raw/patent_assignee.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
patent_assignee = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(patent_assignee_path).repartition(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
patent_assignee.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------------+
|patent_id|         assignee_id|         location_id|
+---------+--------------------+--------------------+
|  9098154|bec1d9ba-7ad6-48c...|ed8be594-09bd-11e...|
|  6241143|6f8a613b-f7a6-499...|ff405b60-cb8e-11e...|
|  8423973|2a04f703-2243-4de...|3fac045d-cb8e-11e...|
|  9731019|8cf738cf-7134-427...|f1b6c93e-cb8e-11e...|
|  5927243|f4b333a6-bc14-460...|fef7fa78-cb90-11e...|
+---------+--------------------+--------------------+
only showing top 5 rows

In [50]:
patent_assignee.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- patent_id: string (nullable = true)
 |-- assignee_id: string (nullable = true)
 |-- location_id: string (nullable = true)

In [51]:
patent_assignee.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7334687

### Drop duplicates based on patent_id

In [52]:
patent_assignee = patent_assignee.dropDuplicates(["patent_id"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
patent_assignee.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7084751

### Write the cleaned patent assignee to S3 

In [54]:
patent_assignee.write.mode("overwrite").parquet(f"{output_path}/patent_assignee.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Cleaning the Patent Inventor Table

In [22]:
patent_inventor_path = "s3://patents-analytics/raw/patent_inventor.tsv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
patent_inventor = spark.read.options(header='True', delimiter='\t', inferSchema='True', escape="\"", multiline='True').csv(patent_inventor_path).repartition(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
patent_inventor.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----------------+--------------------+
|patent_id|      inventor_id|         location_id|
+---------+-----------------+--------------------+
| 10634074|  fl:ry_ln:yuki-1|fda0887b-cb90-11e...|
|  D609230|fl:ja_ln:dejong-7|cf8fb0e1-cb8f-11e...|
|  7470408| fl:da_ln:hesse-8|d9cd82bc-cb8e-11e...|
|  6561731| fl:ja_ln:clare-2|52927c26-cb8e-11e...|
|  D398105| fl:pa_ln:diehl-2|d1b4d389-cb8e-11e...|
+---------+-----------------+--------------------+
only showing top 5 rows

In [25]:
patent_inventor.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

19843047

In [26]:
patent_inventor.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- patent_id: string (nullable = true)
 |-- inventor_id: string (nullable = true)
 |-- location_id: string (nullable = true)

### Drop duplicates based on patent_id

In [64]:
patent_inventor=patent_inventor.dropDuplicates(["patent_id"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
patent_inventor.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8061179

### Write the cleaned patent_inventor to S3

In [66]:
patent_inventor.write.mode("overwrite").parquet(f"{output_path}/patent_inventor.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…